In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from tabulate import tabulate
from gensim.models import Word2Vec
from tabulate import tabulate
# Descargar recursos necesarios de NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

C:\Users\User\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# Cargar y preprocesar los datos
datos = pd.read_csv('datos4v5.csv', encoding='utf-8', delimiter=';')
etiquetas = datos['autor']
textos = datos['texto']

In [3]:
# Preprocesamiento de texto
def preprocesar_texto(texto):
    stop_words = set(stopwords.words('spanish'))
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(texto.lower())
    tokens = [token for token in tokens if token.isalpha()]
    tokens = [token for token in tokens if token not in stop_words]
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return " ".join(tokens)

In [4]:
# Crear el vectorizador TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=1000)

In [5]:
# Preprocesar y vectorizar los textos
textos_limpio = [preprocesar_texto(texto) for texto in textos]
X_tfidf = tfidf_vectorizer.fit_transform(textos_limpio)

In [6]:
# Modelo Word Embedding
model_w2v = Word2Vec(sentences=[word_tokenize(texto) for texto in textos_limpio], vector_size=100, window=5, min_count=1, sg=0)

In [7]:
# Función para obtener vector promedio de Word Embedding
def obtener_vector_promedio(texto, model):
    vectors = [model.wv[word] for word in word_tokenize(texto) if word in model.wv]
    if vectors:
        return np.mean(vectors, axis=0)
    return np.zeros(model.vector_size)  # Asegura que el vector tenga el mismo tamaño que el modelo


In [8]:
# Crear los vectores de Word Embedding
X_w2v = np.array([obtener_vector_promedio(texto, model_w2v) for texto in textos_limpio])

# Asegurarse de que X_w2v tenga las dimensiones adecuadas (2D)
X_w2v = X_w2v.reshape(X_w2v.shape[0], -1)

In [9]:
# Combinar características de TF-IDF y Word Embedding
X_combined = np.hstack((X_tfidf.toarray(), X_w2v))

In [10]:
# Escalar características al rango [0, 1]
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_combined)

# Codificar las etiquetas
label_encoder = LabelEncoder()
etiquetas_encoded = label_encoder.fit_transform(etiquetas)


In [11]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_combined, etiquetas_encoded, test_size=0.3, random_state=42)

In [12]:
# Entrenar y evaluar modelos
modelos = {
    'SVC Lineal': SVC(kernel='linear', probability=True),
    'SVC RBF': SVC(kernel='rbf', probability=True),
    'Random Forest': RandomForestClassifier(n_estimators=100),
    'Regresion Logistica': LogisticRegression()
   
}

resultados = {}

for nombre, modelo in modelos.items():
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    conf_matrix = confusion_matrix(y_test, y_pred)
    resultados[nombre] = {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'Confusion Matrix': conf_matrix
    }

C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning

In [13]:
# Mostrar resultados en una tabla
resultados_df = pd.DataFrame(resultados).T
print(tabulate(resultados_df, headers='keys', tablefmt='grid'))

+---------------------+------------+-------------+----------+-------------------------+
|                     |   Accuracy |   Precision |   Recall | Confusion Matrix        |
+=====================+============+=============+==========+=========================+
| SVC Lineal          |       0.75 |    0.808333 |     0.75 | [[2 0 0 0 0 0 0 0 0]    |
|                     |            |             |          |  [0 3 0 0 0 0 0 0 0]    |
|                     |            |             |          |  [1 0 1 0 0 0 0 0 0]    |
|                     |            |             |          |  [0 0 0 0 0 0 0 1 0]    |
|                     |            |             |          |  [0 0 0 0 2 0 0 0 0]    |
|                     |            |             |          |  [0 1 0 0 0 2 0 0 0]    |
|                     |            |             |          |  [0 2 0 0 0 0 1 0 0]    |
|                     |            |             |          |  [0 0 0 0 0 0 0 2 0]    |
|                     |         

In [126]:
# Cargar nuevo artículo
#Articulo nuevo autor1
#nuevo_articulo = "La resolución de problemas es un tema que en los últimos años se lo aborda con gran interés y preocupación por parte de las instituciones educativa; surge como resultado de considerar el aprendizaje como una construcción social que encierra cálculos, pruebas y contradicciones basado en un proceso creativo y generativo (Coronel & Curotto, 2008). Los profesionales están continuamente expuestos a enfrentar situaciones problemáticas y estresantes que demanda la sociedad actual, él debe ser capaz de procesar información, generar conocimiento útil y tomar decisiones, porque necesita mantener una comunicación asertiva e interdependencia en sus acciones para implicarse con intensidad, energía y responsabilidad en las tareas encomendadas (Arteaga, 2009; Carles, 2001; León, Castaño, Iglesias, & Marugán, 2014; Rigo & Donolo, 2014). Por tanto, es necesario que su formación esté dotada de conocimientos, habilidades y capacidades que lo hagan más competitivo social y profesionalmente. Desempeñarse en el campo laboral y alcanzar metas, implica afrontar y resolver obstáculos que aparecen en el camino. Por lo tanto, debe entrenarse al educando desde las aulas para que aprenda a controlar su aprendizaje y rendimiento que permita desarrollar una madurez actitudinal en su ocupación (Torrano Montalvo & González Torres, 2004). Hay estudios que revelan que su formación debe estar acorde con las necesidades del espacio laboral y que deben saber afrontar y resolver problemas con decisión, eficacia, entrega y compromiso para alcanzar objetivos con éxito (Bados & García, 2014; Torres, 2007; Torres, Badillo, Valentin, & Ramírez, 2014). Ante estos hechos, qué están haciendo las universidades para desarrollar estas competencias en el estudiantado. Lograr estos cambios, requerirá de reformas en el currículum de estudio, donde no solamente se prioricen contenidos disciplinares sino también interdisciplinarios, que tengan un enfoque en competencias con rasgos distintivos, que permitan un desempeño satisfactorio en situaciones reales de trabajo (Moreno T. , 2010). Estos cambios formativos han sido fomentados en los Espacios Europeos de Educación Superior (EEES) para el desarrollo de competencia con un mayor protagonismo del estudiantado en su aprendizaje (González, 2011), en esta misma línea América Latina, ha seguido los pasos en este cambio de estructura formativa en la que incluyen destrezas, conocimientos y contenidos (Proyecto Tuning América Latina, 2011-2013), y Ecuador participa en ella con 12 universidades entre públicas y privadas para fomentar la formación en sus aulas. En esta misma línea, Biggs 2006 citado por Cardeñoso (2012) defiende que el estudiantado universitario, debe implicarse en tareas de aprendizaje de orden superior, y así aprenda a reflexionar críticamente, argumentar y poner en práctica la resolución de casos, problemas o proyectos, esto guarda relación con González y Wagenaar (2006) al mencionar que la competencia de resolución de problemas es una de las competencias más requeridas por los empleadores y graduados en el campo laboral y personal. Con estos antecedentes, se pretende conocer qué competencias de resolución de problemas utiliza el estudiantado universitario, de manera particular en las universidades presenciales de la provincia de Esmeraldas, en las carreras de Ingeniería Mecánica de la Universidad Luis Vargas Torres LVT e Ingeniería en Sistemas Informáticos de la Pontificia Universidad Católica del Ecuador Sede Esmeraldas PUCESE, para analizar los estilos de resolución de problemas que utiliza el estudiantado universitario de acuerdo al género y nivel que cursa. La educación superior en los últimos años ha sufrido procesos de cambios relacionados con la calidad educativa y formativa del profesorado, quien tiene un radio de acción más amplio, porque no solo le corresponde la enseñanza, sino, además, la investigación y otros aspectos que empoderan su rol como académico. Estos cambios en busca de la calidad que está experimentando la universidad, conlleva a modificaciones en las funciones roles y tareas que desempeña el docente, que le exige desarrollar nuevas competencias para desempeñar de manera eficaz sus labores profesionales (Mas, 2011). Ante ello, es necesario que se incorporen mecanismos de formación y evaluación que permita mejorar la calidad docente como parte fundamental del desarrollo profesional y de la identidad docente (Caballero & Bolívar, 2015). La identidad profesional es el conjunto de atributos que caracterizan a un grupo de personas que poseen cierto grado de educación para ejercer un trabajo integral con calidad y compromiso, permitiéndole reconocerse y ser reconocido como un integrante de un gremio, repercutiendo en la vida personal y social del individuo (Caballero 2009; Muñoz & Arvayo 2015). Pues bien, en los académicos, este auto-reconocimiento es conocido como identidad docente; representación que podría mantenerse en el tiempo, mediante contenidos definido a través de roles, creencias y sentimientos que se enlazan al ejercer la profesión en determinados ambientes donde desarrolla su práctica docente (Gómez & Niño 2015). Estos roles del docente en el contexto universitario van a determinar en gran medida las opiniones que desarrollen sobre su profesión y la construcción de una identidad académica (Caballero & Bolívar, 2015), considerada como una identidad específica, donde su resultado es producto de un proceso de socialización entre profesionales que pertenecen a un grupo y se relaciona con el trabajo que se realiza en un ambiente particular (Alonso, Lobato, & Arandia, 2015). Por tanto, es una construcción social que se logra desde que se nace y se va formando, cambiando y modificando, producto de los intercambios que resulta entre actores sociales que participan de un mismo grupo (Salcedo, 2015). Esta transformación involucra los modelos de desempeño de los roles de docentes e investigadores, la forma de comprender la constitución de sus identidades académicas (Larrinaga, 2016), sus pensamientos sobre la enseñanza aprendizaje y la investigación. Es esta doble función de formar e investigar la que da relevancia a la docencia, manteniéndose a la vanguardia de los procesos construidos entre docentes, estudiantes e investigadores (Hernández, 2009). Construir la identidad en los académicos es muy difícil, más aún cuando intervienen profesionales de distintas disciplinas, químicos, médicos, investigador o autores de artículos científicos en revistas de alto impacto (Muñoz & Arvayo 2015). De esta manera, la identidad del profesorado y por ende la del investigador, sería el resultado de varias interacciones entre la experiencia personal y el ambiente social-cultural e institucional dentro del cual se desenvuelve habitualmente el profesional (Aguayo, Castelló & Monereo, 2014). Este trabajo de investigación nace producto de un estudio a nivel de Posgrado realizado en el año 2015 en la Pontificia Universidad Católica del Ecuador Esmeraldas (PUCE-E), concretamente en la Escuela de Contabilidad y Auditoría, donde los resultados revelaron que la función investigadora del profesorado ha sido poco desarrollada generado debilidades en su formación como futuros investigadores (Achilie , 2017); hecho que es subrayado en el estudio realizado por Pazmiño (2016). Por otro lado, los nuevos retos que ha asumido el docente, la investigación, ha traído como efecto su vinculación en diferentes disciplinas o áreas en la que posiblemente no se ha identificado; o ha recibido poca preparación durante su formación (Aguayo, Castelló, & Monereo, 2014). Estos elementos podrían haber generado, nerviosismo, inseguridad en su identidad al desarrollar su tarea docente, especialmente en lo concerniente a sus actividades investigativas. El estudiar la identidad del académico es importante, por cuanto las instituciones de Educación Superior están viviendo procesos de cambio que involucra al docente a asumir nuevos roles, identidades, desarrollar competencias (Cruz, 2015), direccionada al mejoramiento de su perfil profesional; permitiéndole participar en redes de investigación, asesorías, difundir su producción científica y vincularse hasta en procesos de gestión en la universidad (Trujillo, 2015). Además, este estudio daría pautas a la PUCE-E para que busque alianzas estratégicas con otras universidades a nivel local, nacional e internacional, con la finalidad de establecer nuevos planes de formación y desarrollo profesional, que conlleve a seguir fortaleciendo la actividad investigadora; más aún cuando el Plan Nacional del Buen Vivir (2013) en su cuarto objetivo establece los lineamientos estratégicos encaminados al mejoramiento de la educación. Asimismo, la LOES (2010) en su art 13 literal d, señala la necesidad de fortalecer el desarrollo de la docencia y la investigación en todos los niveles y modalidades del sistema superior, lo que es subrayado en la misma Ley en su art 6 literal a. Además, podría ser beneficioso asegurar la incorporación de profesionales que motiven el interés entre otros aspectos, por la investigación, indicador de productividad para la universidad, que a la vez posibilita dar pautas para tener un acercamiento con los docentes que permita resolver la interrogante ¿Cómo se identifica el profesorado en su rol investigador? Basados en estos antecedentes y en cuanto pueda aportar a la institución, específicamente a la Escuela de Contabilidad y Auditoría de la PUCESE, se planteó como objetivo, describir la identidad del docente en su rol de investigador. Los desastres naturales han estado ligados en la historia de la humanidad por su poder destructivo y efectos negativos perdurables en la sociedad  (Espinoza, 2008).  Los ecuatorianos hace pocos años experimentaron estos eventos con el sismo de abril 16 de 2016.  Las provincias que sufrieron mayores afectaciones fueron Manabí y Esmeraldas;  en  el caso de esta última, se evidenció que los mayores daños sociales, económicos y materiales fueron en la Isla del cantón Muisne. Las familias de la isla tuvieron que dejar sus viviendas, negocios y todo su diario vivir para albergarse en zonas de ayuda gubernamental. Estos eventos dejan daños en las viviendas que no siempre cuentan con el apoyo ni el financiamiento del estado para sus reparaciones, en muchos de los casos deben ser financiados con recursos propios de los afectados (Maldonado & Albio, 2013). En cuanto a las actividades del comercio en Muisne, las tiendas de abarrotes son las que dan mayor generación económica.  Al mismo tiempo, fueron las más afectadas por cuanto se redujeron sus ventas debido a que las familias estaban albergadas en centros de ayuda del estado. Además de ello, el mal estado de las carreteras que dificultaban el acceso de los proveedores con las provisiones.     Aunado a esto, un estudio sobre secuelas del sismo de 2016 desarrollado por Ruiz (2017),  muestra  el cambio del comportamiento en compras un año después del terremoto en una empresa dedicada al comercio menor.  Se pudo observar que los clientes de Manabí redujeron sus compras en un 29% en relación con el año anterior; y que, además el 33% de los clientes que compraron en el periodo antes del terremoto no volvieron a comprar después del mismo.  Es por ello necesario que se dé apoyo a las pequeñas y medianas empresas que en muchos casos no pueden retomar sus actividades después de estas catástrofes  (Dresdner & Sehnbruch, 2010; Mella, 2012).    Estos desastres,  inmovilizan temporalmente la economía, puesto que la destrucción en infraestructura y equipamiento del sector productivo, imposibilitan su funcionamiento  (Rasse & Letelier,  2012; Razeto,  2013).  Hay países como es el caso de Chile que han experimentado en repitas ocasiones estos eventos, uno a citar es el del 27 de febrero de 2010 en el que desarrollaron el Plan de Reconstrucción Terremoto y Maremoto para apoyar especialmente a Pymes y pescadores artesanales con la creación de incentivos tributarios y financieros para la reconstrucción de infraestructura y equipamiento de las zonas más afectadas por el sismo  (Gobierno de Chile,  2010). Además de ello, se puede citar el Programa de Desarrollo Solidario (PDS) del Banco del Desarrollo que se lleva a cabo en Chile y brinda apoyo financiero y formativo a más de 10.000 mujeres de bajos recursos y aproximadamente el 60% de ellas los destinan al mejoramiento de sus viviendas (Rojas, 2017), todo esto con el fin de poner en práctica alternativas de recuperación de las zonas afectadas.    En  cuanto al  gobierno ecuatoriano  luego de  sismo,  planteó  políticas,  programas  y estrategias de reactivación económica de la población  afectada,  a  través  de  la  Ley Orgánica  de  Solidaridad  y  de Corresponsabilidad  Ciudadana  para  la Reconstrucción  y  Reactivación  (LOSCCRR) de  las  zonas  afectadas,  a  través  de  la contribución  solidaria  sobre  la remuneración,  el  patrimonio,  bienes inmuebles,  utilidades,  exoneración  y/o devolución  del  anticipo  de  impuesto  a  la renta,  incremento  de  tarifa  IVA  al  14%, excepto a las zonas afectas, y la exoneración del pago de cuotas de Régimen Simplificado RISE (LOSCCRR, 2016).    De acuerdo con informes del Plan Reconstruyo Ecuador,  las  mayores afectaciones se concentraron en las empresas grandes del sector formal en un 43,40% y las empresas pequeñas en 31,10%  (Comite de Reconstrucción y Reactivación Productiva, 2016). Además, las gestiones de reactivación se dieron también en el sector financiero como es el caso del Ban Ecuador, que realizó aproximadamente 2.251 operaciones de crédito en beneficio de los pobladores de Muisne a partir del 9 de mayo de 2016, con un monto superior a USD 6.5 millones destinados a las distintas actividades productivas de la zona  (Ban Ecuador, 2017).     Luego de aproximadamente un año y medio de este evento natural,  se  ha considerado pertinente analizar los beneficios a los que accedieron los comerciantes de víveres del cantón Muisne, en virtud de los incentivos promovidos por la LOSCCRR. Desde inicios del siglo XXI, las Instituciones de Educación Superior, así como los docentes universitarios,  están  viviendo un proceso de transformación en sus diferentes dimensiones, debido a varios cambios a nivel social, cultural, educativos, tecnológicos económicos y políticos. En este marco, el docente universitario ecuatoriano ha encontrado nuevos y variados retos y desafíos, entre ellos la investigación. Esta actividad ha sido un elemento importante debido a que conduce a la generación de nuevos conocimientos, como lo concibe la Organización de las Naciones Unidas para la Educación, la Ciencia y la Cultura (UNESCO, 2005) y en los últimos años, ha asumido un rol protagónico (Segura, 2008). Además, esta misión ha generado debates importantes, en el caso de la academia ecuatoriana, debido  a las nuevas obligaciones en la actividad del docente (Cortés, 2006, p. 9).Así, dentro de las actividades de investigación que han sido consideradas de envergadura para el fortalecimiento y cambio de las universidades ecuatorianas1, la producción científica ha sido el factor que formado parte de los debates en los ámbitos políticos, académicos, culturales, sociales, entre otros, de forma significativa,  esto teniendo  en cuenta que en  el contexto global latino  e iberoamericano la  producción  científica ecuatoriana  es baja  y  poco visible  internacionalmente (Bruque, 2013).  La investigación científica, como uno de los motores de la academia ecuatoriana, de hecho, ha formado parte de las políticas transformadoras del país. Es así que, el Plan Nacional del Buen Vivir 2013-2017, ha definido entre sus cambios más importantes: impulsar la transformación de la matriz productiva, articulando la investigación científica, tecnológica y la unión de la Educación Superior con el sector productivo, para conseguir una mejora permanente de la productividad y la competitividad de manera integral. En este contexto, varias universidades del país han comenzado procesos de auditorías (internas y externas) para conocer sus capacidades, fortalezas y debilidades en los distintos aspectos de la academia: docencia, gestión e investigación. Una de estas universidades ha sido la Pontificia Universidad Católica del Ecuador Sede Esmeraldas (PUCESE), que al ser evaluada por el anterior Consejo  Nacional  de  Evaluación  y  Acreditación  de  la  Educación  Superior  (CONEA,  2010), mostró  un  escenario deslucido  en  la producción científica,  con  un 12,07%  en  relación a  los otros indicadores (academia, gestión  y extensión), y con este porcentaje la Sede Esmeraldas no alcanzó los índices necesarios  en investigación de acuerdo al número de sus docentes. Sobre esta problemática,  varios han sido los aportes que se han enfocado en la producción investigativa. Así, por ejemplo, existen investigaciones que han analizado la opinión del docente universitario en contabilidad en relación a su producción científica y las causas que pueden explicar su diferencia respecto a otras disciplinas (Escobar et al., 2014). Otros estudios se han enfocado en conocer el estado de la investigación de las ciencias económicas, administrativas y contables en relación a otras áreas de estudio (Flores, 2011), así como también, las dificultades de esta área en la producción investigativa (Briceño et al., 2007). Asimismo, existen investigaciones que han valorado el papel del departamento de postgrado en la formación de investigadores contables (Viloria, Casal, Peña, 2010).Empero,  en  el  Ecuador,  los  estudios  sobre  esta  problemática  han  sido  escasos  y  se  han enfocado sobre todo  en  analizar de manera  bibliométrico-comparativa  la  producción científica del país, exponiendo que en el contexto global latino e iberoamericano la producción científica ecuatoriana es baja y poco visible internacionalmente (Bruque, 2013). Así,  esta  investigación tiene como objetivo analizar los elementos que contribuyen o limitan la producción investigativa de la PUCESE, en particular, de los docentes de la Escuela de Contabilidad y Auditoría.El Plan de  gobierno  ecuatoriano realiza  reformas  en  la  administración  de tributos para financiar la educación, la salud y  la  seguridad  social  (Paz  &  Miño,  2015). Estos cambios traen  consigo  el  seguir fomentando la cultura tributaria en todos los niveles. De acuerdo al Ministerio de Finanzas (2015) los  impuestos  representan  el 63.76%  de  los  ingresos  permanentes, siendo  los  de  mayor  recaudación  el impuesto a la renta, seguido del impuesto al valor  agregado  (IVA).  Este  último  modelo fiscal tiene dos tipos de contribuyentes, los clasificados  en  el  Registro  Único  de Contribuyentes (RUC) y los  que conforman el  sistema  de  Régimen  Impositivo Simplificado (RISE), en el  que  se cancelan cuotas  mensuales  que  reemplazan  la obligación de realizar declaraciones de IVA e impuesto a la renta. (SRI, 2010)  La  presencia  del  RISE  desde  el  año 2008  ha  permitido  que  algunos  sectores como  el  comercio  y  servicio  puedan contribuir  a  la  formalización  de  los pequeños  negocios  y  a  su  crecimiento, mediante  el  alivio  de  la  carga  burocrática que  enfrentan,  y  potenciar  sus  beneficios” (Granda  &  Zambrano  ,  2012,  p.  13).  Esa experiencia ya  la  han  vivido  otros  países, como  Argentina  con  el  Monotributo,  en  el que según Salim y D’Angela (2006), tuvo éxito  porque  permitió  el  incremento  en  la recaudación, al ser alícuotas fijas; pero a la vez  mencionan  que  se  ha  perdido participación  en  relación  a  la  recaudación total. Otro caso  a citar es  el de Colombia, donde Sabaíni y Morán (2016) insisten en la necesidad de buscar alternativas en materia de  IVA,  que  regulen  su  recaudación  a  los sectores más vulnerables y que se lo podría relacionar  con  lo  que  indica  Rodríguez (2014)  sobre  la  necesidad  de  que  estos cambios  no  deben  caer  en  estimaciones presuntivas,  que  pongan  en  riesgo  la equidad  tributaria  entre  contribuyentes; además,  Granda  y  Zambrano  (2012) comentan que existe mucha desinformación sobre el RISE en relación a los beneficios y obligaciones que la ley determina.  Al  ser  el  pago  de  tributos  una obligación  de  todo  contribuyente,  fue necesario  buscar  un  sector  que  concentre comerciantes  con  diferentes  modelos fiscales,  como  es  el  caso  del  mercado municipal  y  se  investiguen  qué  problemas tributarios pueden tener, ya que este sector agremia  a  107  comerciantes  que contribuyen  con  el  dinamismo  de  la economía de la ciudad y que a la vez dieron la  pauta  para  que  esta  investigación  se plantee  como  objetivo  el  analizar  la situación  tributaria  de  un  grupo  de comerciantes de legumbres y productos de consumo masivo  del  Mercado  Municipal de la ciudad de Esmeraldas."

#Articulo en base de datos autor10 - si esta en la base de datos
#nuevo_articulo="En esta investigación se elaboró un generador de código utilizando el paradigma de líneas de producto software junto con otras metodologías de desarrollo por analogía. En la actualidad las aplicaciones de desarrollo se enfocan a un ambiente web y requieren de seguridad, mensajería, control de usuarios, etc., por lo que cada día se vuelven más complejas. Una línea de producto software ofrece una alternativa para hacer frente a esa complejidad proporcionando herramientas que reutilicen el código y se mejore el desempeño, la calidad y el tiempo en el desarrollo de un producto software. El objetivo principal de este trabajo es dar a conocer las ventajas y utilidad potencial de la línea de productos software construido por analogía y la mejora en el desarrollo de una aplicación web Java EE. El generador pretende automatizar el proceso de codificación de las anotaciones de persistencia e implementación de los métodos con Java Persistence API (JPA) del Data Access Object (DAO) de la Entidad para el acceso a la base de datos. El prototipo fue desarrollado en lenguaje de programación Ruby, librerías Gtk2 para el entorno gráfico, ERB para el manejo de archivos de texto a través de un lenguaje de plantillas. Esta herramienta aumenta el desempeño de desarrollo y una disminución del tiempo de codificación comprobado por la estimación por intervalo que se realizó a un grupo de programadores. .La Mejora de Procesos Software (MPS) o (SPI) con sus siglas en ingles es un concepto relativamente nuevo, la mayoría de sus ideas, conceptos y métodos se han adoptado de ladefinición de calidad del sistema de fabricación, la teoría se ha formado a partir de autores del área de calidad (Brenda, Mirna, & Jezreel, 2013). Un proceso implica el uso de los recursos de una organización, para obtener algo de valor. Así, ningún producto puede fabricarse y ningún servicio puede suministrarse sin un proceso, y ningún proceso puede existir sin un producto o servicio, se define a un proceso como, un conjunto de actividades o tareas que se relacionan entre sí, y que se ejecutan siguiendo un orden lógico con el propósito de alcanzar un resultado específico a partir de las entradas de recursos e información. Los procesos constituyen uno de los principales problemas dentro de las organizaciones productoras de software, que impiden el incremento de la productividad y la calidad, de ahí la necesidad de trazar estrategias para mejorarlos (“Proceso de mejora de Software - EcuRed,” n.d.-a). La palabra mejora está vigente en diferentes esferas de la vida, sobre todo en ambientes empresariales donde ha dejado de ser una opción para convertirse en una imperiosa necesidad. Mejora proviene del verbo mejorar que implica cambiar, El Diccionario Enciclopédico define mejora como: “Acción o efecto de mejorar”. “Cambio hecho en una cosa, por lo que resulta mejorada”. Según el diccionario de la Real Academia Española (RAE), mejora es: “Medra, adelantamiento o mejora de algo”. Haciendo un análisis de los significados enunciados y adaptándolos a la presente investigación, se define que mejora no es más que mejorar algo haciéndolo pasar a un estado superior. Por lo que la podemos definir como La mejora del Proceso Software es un mecanismo de mejora continua de la calidad. Se puede utilizar para elevar capacidad de los contratistas, para auditar desarrollos de software interno y para planificar la estrategia de ingeniería del software de la empresa. Además tenemos otra definición “La Mejora del Proceso Software es un mecanismo de mejora continua de la calidad. Se puede utilizar para evaluar la capacidad de los contratistas, para auditar desarrollos software internos y para planificar la estrategia de ingeniería del software de la empresa” (Cuevas Agustín, Gonzalo, 2002)."

#Articulo nuevo - autor desconocido
#nuevo_articulo="El uso de computadores como herramienta de aprendizaje en la educación fue propuesto por Seymour Papert en 1980, convirtiendo el aprender-pensar en construir-programar, a esto lo denominó Construccionismo. El Instituto de Tecnología de Massachusetts, en 1998 construyó robots educativos LEGO MINDSTORMS como recurso de aprendizaje, con ello se logró vincular a jóvenes con la tecnología, esta herramienta sirvió para evaluar la trasferencia de conocimiento y trabajo en equipos de estudiantes de educación básica utilizando rúbricas para la obtención de datos. El acuerdo ministerial 041-14, propone introducir en la malla curricular de Educación General Básica (EGB), tres horas de clases destinadas a clubes orientadas al campo de acción científico, interacción social y vida práctica, este acuerdo sirvió de base en la presente investigación. La presente investigación desarrollada bajo el título “Dispositivo de monitoreo basado en el internet de las cosas que obtenga evidencias de lo que acontece en el barrio 20 de noviembre de la ciudad de esmeraldas y genere una alarma de alerta en el mismo”, propone desarrollar una solución tecnológica que evidencie hechos delincuenciales en el sector utilizando dispositivos electrónicos, debido a los diferentes acontecimientos de vandalismo que atentan a la integridad de los moradores, por lo cual fue necesario utilizar un microprocesador acorde a las necesidades de la investigación, así como, un medio web para la comunicación entre los usuarios del dispositivo. Esmeraldas en las últimas décadas ha sufrido un aumento de la delincuencia que mantiene preocupada a su población; cada año el aumento o disminución de la delincuencia varía según aplicativos que se tomen en cuenta para mantener segura la población, pero a pesar de ello la delincuencia aún permanece en la ciudad. El Ministerio del Interior (2017) expone que el crimen organizado, homicidios intencionales, robo a domicilio, personas, unidades económicas, carros, motos, tráfico y microtráfico de drogas son los acontecimientos delictivos más notorios en la ciudad de Esmeraldas a cualquier hora del día. La falta de empleo, explotación del empleado no profesional y pagos irrisorios por el desempeño laboral son puntos que han favorecido al aumento de la delincuencia como tal. Un ejemplo estadístico de hace 10 años donde Diario el Universo (2008) publicó que la Policía Nacional desde el 2008 comentó que la ciudad había aumentado el nivel de delincuencia en un 15% tras el asesinato de un joven policía en la calle Malecón, una cifra muy alarmante para la pequeña población que mantiene en incertidumbre a más de un ciudadano. Años más tarde tras varias intervenciones por parte de la policía nacional ese porcentaje que en un principio iba en aumento se vuelve menos alarmante para la zona urbana del cantón, a continuación, en la figura 1 se muestran datos estadísticos que proporciona el Ministerio del Interior (2017) a los ciudadanos para analizar la variabilidad de la delincuencia año tras año. La figura 1 representa el 31.18% de disminución de robos a domicilio entre el año 2017 y el año 2016, siendo así que se suscitaron 417 robos a domicilio en 2016 y 287 robos a domicilio en 2017, en donde los días con mayor recurrencia para delinquir serían miércoles, viernes y domingos en horas de la madrugada, como se muestra en la figura 2. Por otra parte, fomentar el deporte, los distintos tipos de arte, el descubrir talentos desde la niñez no es algo a lo que se le dé mucha importancia en la ciudad, iniciativa que debería ser tomada por las autoridades competentes del asunto social en Esmeraldas. Un dato curioso es que la provincia es cuna de deportistas (como coloquialmente se le llama) que participan en diferentes disciplinas deportivas en el Ecuador y aún con estos antecedentes no se realizan los procesos adecuados para involucrar a los jóvenes y adolescentes de una población mayormente caracterizada por su alto rendimiento físico en los diferentes programas atléticos que pueden existir. En la figura 3 se pueden observar varias acciones que se pueden tomar para disminuir la delincuencia o inseguridad en la ciudad, encuesta publicada por CEDATOS basado en la investigación por parte de la ESPOL. La falta de Dios, sus principios, mandamientos, sacramentos hacen que mayormente la juventud se envuelva en círculo vicioso de perversidad y desadaptación total de la sociedad, trayendo como consecuencia un déficit de valores, moral, integridad y por su puesto falta respeto y temor ante Dios, donde atentar contra el prójimo es un acto común. El barrio “20 de Noviembre” no es la excepción en el tema, ya que también se suscitan hechos de tipo delincuencial originados mayormente por dos grupos contrincantes de jóvenes y adolescentes de acuerdo a versiones de los moradores del sector. La mayoría de involucrados en estos grupos de jóvenes y adolescentes no asisten a un establecimiento educativo lo que origina que tengan tiempo de ocio y dedicación a actividades delincuenciales, lastimando su integridad y la de la comunidad en general. El barrio carece de zonas de recreación, de programas o actividades que incluyan a estos jóvenes y adolescentes, ya que la falta de estas actividades le da espacio a que los vicios se adueñen de estos jóvenes. Lanzamientos de objetos peligrosos, detonaciones de armas de fuego, robos, atracos a transeúntes y conductores son varios de los acontecimientos que se suscitan en el barrio perjudicando una vez más la integridad de los moradores y la de sus propiedades privadas. Todo este ambiente trae preocupación y temor a los moradores del barrio, ya que no existen cámaras de seguridad o un dispositivo tecnológico que puedan captar todas estas anomalías y presentarlas como evidencia de actos de índole delincuencial ante la Policía Nacional del Ecuador para colocar la respectiva denuncia. La incorporación de las TIC en los procesos educativos, en la actualidad, es indiscutible y son varias las herramientas tecnológicas desarrolladas con el fin de apoyarlos, sin embargo, ninguna herramienta tecnológica, por sí sola, puede aportar a la educación sin estar ligada a una estrategia didáctica apropiada. Para investigadores como Oyola y Gómez (2012), Ferro y Martínez (2009), Castillo (2008), Mora (2003), entre otros, el proceso de enseñanza aprendizaje se ve fortalecido con el empleo de estrategias didácticas que garantizan el uso apropiado de las tecnologías como recursos educativos. Dentro de la dinámica pedagógica, el docente, desde una postura constructivista, debe promover procesos educativos que permitan la participación de los estudiantes a través de actividades previamente planificadas y diseñadas para propiciar su desarrollo intelectual (Castro, Peley y Morillo, 2006). En este sentido, es primordial que el docente considere las TIC como un recurso de apoyo que debe insertar al delinear su estrategia didáctica. Para Ocampo (2003), las estrategias didácticas son procesos o secuencias de actividades que sirven de base a la realización de tareas intelectuales y que se eligen con el propósito de facilitar la construcción, permanencia y transferencia de conocimientos (p. 9). Por otro lado, Carrasco (2004), define una estrategia didáctica de enseñanza como el conjunto de instrucciones que el docente diseña y aplica para beneficiar los procesos de enseñanza-aprendizaje Bajo este contexto, según Díaz y Hernández (2012), las estrategias didácticas de enseñanza pueden ser: pre-instruccionales, co-instruccionales y post-instruccionales; las que están respectivamente ligadas a diferentes momentos de una clase: inicio, en el que se plantea lo que se va a aprender y cómo se lo hará; desarrollo, en el que se exponen los contenidos y se aplican técnicas motivacionales para mantener el interés de los estudiantes; cierre, en el que se comprueba si los objetivos propuestos al inicio de la clase se han alcanzado. En el campo de las matemáticas, la enseñanza-aprendizaje debe estar en concordancia con el avance tecnológico, para lo cual, debe estar ligada a una metodología que favorezca el desarrollo cognitivo de los estudiantes, que estimule su pensamiento crítico y su autorregulación, que tienda el puente entre la educación tradicional y aquella que emplea la tecnología a su favor (García, Escalera y Navarro, 2011). En el entorno educativo, varios son los recursos tecnológicos que se utilizan, entre ellos: software educativo, simuladores, herramientas multimedia y de presentación, correo electrónico, blogs, etc., algunos de ellos creados específicamente como recursos didácticos y otros que se han adaptado. En concordancia a lo anteriormente expuesto, Herrera, Novelo, Díaz y Hernández (2016), opinan que dentro de las TIC, los software educativos especializados también son herramientas que se pueden tomar como estrategias para la enseñanza-aprendizaje de las matemáticas (p. 8). De igual manera, Morales y Peña (2013) argumentan que el empleo de herramientas tecnológicas educativas para la enseñanza de matemáticas puede ser fructífero si están acompañadas de estrategias apropiadas. En razón de lo mencionado, el uso de cualquiera de estos recursos no es una garantía de que el proceso educativo se lleve adecuadamente y que su efectividad se vea reflejada en los logros alcanzados por los estudiantes; de ahí que es necesario que el docente planifique su clase y establezca la herramienta tecnológica que, de acuerdo con sus características, se adapte mejor a su estrategia didáctica. Para este proyecto se considera como unidad de estudio el capítulo de lógica matemática, abordando los siguientes temas: proposiciones, conectores lógicos, lectura y escritura lógicas contemplada en el syllabus de Matemática de primer nivel de la Escuela de Sistemas de la PUCE Esmeraldas. Con esta referencia se diseña la estrategia didáctica con un enfoque constructivista, considerando los tres momentos de una clase: inicio, desarrollo y cierre; y contemplando una evaluación tanto formativa como sumativa. En base a los contenidos y actividades a realizar, se considera pertinente emplear como recurso digital de apoyo un software educativo, que permita la creación y manejo de contenidos, que además resulte ser amigable para el usuario; de acceso libre y gratuito. En función de estos requerimientos y tras un análisis previo, se considera que la herramienta tecnológica idónea para apoyar la estrategia es el software educativo eXe-Learning. Una vez estructurada la estrategia didáctica apoyada con el recurso digital eXe-Learning es importante determinar si su aplicación produce algún efecto en el proceso de enseñanza-aprendizaje de los estudiantes, para ello se plantea la siguiente hipótesis: La media de las calificaciones del test aplicado a los estudiantes, que recibieron las clases con la estrategia apoyada con eXe-Learning es mayor que la media de las calificaciones de aquellos que recibieron las clases sin la estrategia apoyada con eXe-Learning. A continuación, se detalla la metodología empleada, el diseño de la estrategia apoyada con el eXe-Learning, los resultados obtenidos al aplicarla y finalmente se exponen las conclusiones generadas por esta investigación."

#Articulo nuevo - autor desconocido
#nuevo_articulo="Las lámparas LED son un instrumento utilizado para diversos trabajos dentro de la profesión de odontología. Dentro de la rama de la odontología estética se utiliza estas lámparas para el tratamiento de blanqueamiento dental; pero la gente que requiere este tratamiento exige el uso de estas lámparas sin tener el conocimiento de que esto les puede afectar de una manera negativa al estado de sus piezas dentales. Objetivo: Examinar los efectos de las lámparas LED en el aclaramiento dental en las piezas dentarias. Metodología: Para buscar instruir a todo lector, este trabajo utilizó el método analíticosintético; ya que se requirió el buscar y leer diversos textos analizándolos de manera científica y sintetizando los datos más importantes de todas las investigaciones y trabajos consultados. Resultado: El utilizar la lámpara LED como medio para activar el gel blanqueador termina en que el paciente experimente un grado mayor de sensibilidad dental que puede llegar a ser permanente. Conclusiones: El blanqueamiento dental se lo puede realizar los pacientes sin requerir el uso de las lámparas LED y llegar al mismo resultado sin correr el riesgo de adquirir o aumentar el grado de sensibilidad dental de su boca.Las lámparas LED son un instrumento utilizado para diversos trabajos dentro de la profesión de odontología. Dentro de la rama de la odontología estética se utiliza estas lámparas para el tratamiento de blanqueamiento dental; pero la gente que requiere este tratamiento exige el uso de estas lámparas sin tener el conocimiento de que esto les puede afectar de una manera negativa al estado de sus piezas dentales. De manera global, toda la gente se está empezando a preocupar cada vez más por la estética de como ellos se ven frente a otras personas, por lo cual buscan mejorar cada aspecto de su cuerpo; como es el caso del blanqueamiento dental, la ortodoncia, placas o implantes dentales. Para tener la información manejada de la mejor manera, se utiliza la ficha de recolección de datos; para tener un acceso rápido a todos los artículos que se revisarán a lo largo de la investigación. De esta manera se busca responder a la duda de que si estos tratamientos tienen algún efecto negativo en la vida de las personas; como es el caso del tema de la presente investigación, que busca identificar los efectos de utilizar una lámpara LED en el tratamiento de blanqueamiento dental. Esta investigación se divide en 4 capítulos, los cuales se distribuyen la información de una manera específica. En el primer capítulo encontramos el planteamiento del problema con sus consiguientes preguntas relacionadas al tema, planteando así también el objetivo general y los específicos. Como capítulo 2 tenemos la información del tema propiamente dicha expuesta en el marco teórico, en el cual se explican la historia de la lámpara LED, como se inventó y su evolución. Una breve historia del blanqueamiento dental y como las lámparas influyen en los resultados de estos tratamientos."


In [127]:
# Preprocesamiento del nuevo artículo
nuevo_articulo_limpio = preprocesar_texto(nuevo_articulo)

In [128]:
# Crear perfil del nuevo artículo
nuevo_articulo_tfidf = tfidf_vectorizer.transform([nuevo_articulo_limpio])
nuevo_articulo_w2v = obtener_vector_promedio(nuevo_articulo_limpio, model_w2v)
nuevo_articulo_combined = np.hstack((nuevo_articulo_tfidf.toarray(), nuevo_articulo_w2v.reshape(1, -1)))
# Escalar características al rango [0, 1]
nuevo_articulo_scaled = scaler.transform(nuevo_articulo_combined.reshape(1, -1))

In [129]:
# Crear la tabla para almacenar los resultados
resultados_nuevo_articulo = []
    
# Realizar predicción para el nuevo artículo
for nombre, modelo in modelos.items():
    autor_predicho = modelo.predict(nuevo_articulo_scaled)
    probabilidad_prediccion = np.max(modelo.predict_proba(nuevo_articulo_scaled))
    autor_predicho_decodificado = label_encoder.inverse_transform([autor_predicho])[0]
    resultados_nuevo_articulo.append([nombre, autor_predicho_decodificado, probabilidad_prediccion])



C:\Users\User\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:153: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:153: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:153: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:153: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_

In [130]:
# Mostrar resultados en una tabla
print("\nNuevo Artículo:")
print(tabulate(resultados_nuevo_articulo, headers=['Modelo', 'Autor Predicho', 'Probabilidad'], tablefmt='grid'))


Nuevo Artículo:
+---------------------+------------------+----------------+
| Modelo              | Autor Predicho   |   Probabilidad |
+=====================+==================+================+
| SVC Lineal          | autor7           |       0.4613   |
+---------------------+------------------+----------------+
| SVC RBF             | autor2           |       0.19235  |
+---------------------+------------------+----------------+
| Random Forest       | autor10          |       0.31     |
+---------------------+------------------+----------------+
| Regresion Logistica | autor7           |       0.246075 |
+---------------------+------------------+----------------+
